### 搭建基本模块--神经元

In [3]:
import numpy as np

def sigmoid(x):
    # our activation function: f(x) = 1 / (1 * e^(-x))
    return 1 / (1 + np.exp(-x))

class Neuron():
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        
    def feedforward(self, inputs):
        # weight inputs, add bias, then use the activation function
        total = np.dot(self.weights, inputs) + self.bias
        return sigmoid(total)
    
weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4
n = Neuron(weights, bias)

# inputs
x = np.array([2, 3])   # x1 = 2, x2 = 3
print(n.feedforward(x)) # 0.9990889488055994

0.9990889488055994


### 搭建神经网络
- 这个网络有2个输入、一个包含2个神经元的隐藏层(h1和h2)，包含1个神经元的输出层o1
- 我们假设上面的网络里所有的神经元都具有相同的权重w=\[0,1\]，和偏置b=0，激活函数都是sigmoid，那么输出结果是什么呢？

In [5]:
class OurNeuralNetworks():
    """
    A neural network with:
      - 2 inputs
      - a hidden layer with 2 neurons (h1, h2)
      - an output layer with 1 neuron (o1)
    Each neural has the same weights and bias:
      - w = [0, 1]
      - b = 0
    """
    def __init__(self):
        weights = np.array([0, 1])
        bias = 0
        
        # The Neuron class here is from the previous section
        self.h1 = Neuron(weights, bias)
        self.h2 = Neuron(weights, bias)
        self.o1 = Neuron(weights, bias)
        
    def feedforward(self, x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)

        # The inputs for o1 are the outputs from h1 and h2
        out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))
        return out_o1
        
network = OurNeuralNetworks()
x = np.array([2, 3])
print(network.feedforward(x)) # 0.7216325609518421


0.7216325609518421


### 训练神经网络
- 在训练神经网络之前，需要一个标准定义它到底好不好，以便我们进行改进，这就是**损失**。
- 比如使用均方误差（MSE）来定义损失，即所有数据方差的平均值。预测结果越好，损失就越低。
- 训练神经网络就是将损失最小化。

#### 计算损失函数的代码如下：

In [7]:
def mse_loss(y_true, y_pred):
    # y_true and y_pred are numpy arrays of the same length
    return ((y_true - y_pred) ** 2).mean()

y_true = np.array([1, 0, 0, 1])
y_pred = np.array([0, 0, 0, 0])

print(mse_loss(y_true, y_pred)) # 0.5

0.5


#### 减少神经网络损失
- 我们知道，改变网络的权重和偏置可以影响预测值，但是应该怎么做呢？
- 预测值是由一系列网络权重和偏置计算出来的，所以损失函数实际上是包含多个权重、偏置的多元函数：L(w1, w2, w3, w4, w5, w6, b1, b2, b3)。
- 如果调整一下w1，损失函数是会变大还是变小？我们需要知道**L**对**w1**的偏导数是正是负才能回答这个问题。
- 根据链式求导公式计算**L**对**w1**的偏导数，这种向后计算偏导数的系统称为反向传播（backpropagation）。
- 带入实际数据计算得到的结果告诉我们：如果增大**w1**，损失函数**L**会有一个非常小的增长。

#### 随机梯度下降（SGD）
- 经过前面的运算，我们已经有了训练神经网络所有数据。接下来该如何操作呢？
- SGD定义了改变权重和偏置的方法${w_1 \leftarrow w_1 - \eta\frac{\partial L}{\partial w_1}}$，${\eta}$是一个常数，称为学习率。
- 如果我们用这种方法去逐步改变网络的权重**w**和偏置**b**，损失函数会缓慢地降低，从而改进我们地神经网络。
- 训练流程如下：
    - 从数据集中选择一个样本；
    - 计算损失函数对所有权重地偏置地偏导数；
    - 使用更新公式更新每个权重和偏置；
    - 回到第1步。
- 用代码实现这个过程：

In [9]:
def sigmoid(x):
    # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
    fx = sigmoid(x)
    return fx * (1 - fx)

def mse_loss(y_true, y_pred):
    # y_true and y_pred are numpy arrays of the same length
    return ((y_true - y_pred) ** 2).mean()

class OurNeuralNetwork():
    """
    A neural network with:
      - 2 inputs
      - a hidden layer with 2 neurons (h1, h2)
      - an output layer with 1 neuron (o1)
      
    *** DISCLAIMER ***
    The code below is intend to be simple and educational, NOT optimal.
    Real neural net code looks nothing like this. Do NOT use this code.
    Instead, read/run it to understand how this specific network works.
    """
    def __init__(self):
        # weights
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()
        # biases
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()
        
    def feedforward(self, x):
        # x is a numpy array with 2 elements, for example [input1, input2]
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1
    
    def train(self, data, all_y_trues):
        """
        - data is a (n x 2) numpy array, n = # samples in the dataset.
        - all_y_trues is a numpy array with n elements.
        Elements in all_y_trues correspond to those in data.
        """
        learn_rate = 0.1
        epochs = 1000 # number of times to loop through the entire dataset
        
        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # Do a feedforward (we'll need these values later)
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)
                
                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)
                
                sum_o1 = self.w5 * x[0] + self.w6 * x[1] + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1
                
                # - - - Calculate partial derivatives.
                # - - - Naming: d_L_d_w1 represents "partial L / partial w1"
                d_L_d_ypred = -2 * (y_true - y_pred)
                
                # Neuron o1
                d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
                d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
                d_ypred_d_b3 = deriv_sigmoid(sum_o1)
                
                d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
                d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)
                
                # Neuron h1
                d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
                d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
                d_h1_d_b1 = deriv_sigmoid(sum_h1)
                
                # Neuron h2
                d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_w4 = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_b2 = deriv_sigmoid(sum_h2)
                
                # - - - update weights and biases
                # Neuron h1
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1
                
                # Neuron h2
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2
                
                # Neuron o1
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3
                
            # - - - Calculate total loss at the end of each epoch
            if epoch % 10 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f", (epoch, loss))
                
# Define dataset
data = np.array([
    [-2, -1], # Alice
    [25, 6],  # Bob
    [17, 4],  # Charlie
    [-15, -6] # diana
])
all_y_trues = np.array([
    1, # Alice
    0, # Bob
    0, # Charlie
    1 # diana
])

# Train our neural network!
network = OurNeuralNetwork()
network.train(data, all_y_trues)

Epoch %d loss: %.3f (0, 0.4588985952447696)
Epoch %d loss: %.3f (10, 0.4559672713951668)
Epoch %d loss: %.3f (20, 0.45225347378751546)
Epoch %d loss: %.3f (30, 0.44768477381050636)
Epoch %d loss: %.3f (40, 0.44221828512636424)
Epoch %d loss: %.3f (50, 0.4358435174082699)
Epoch %d loss: %.3f (60, 0.42857310108322844)
Epoch %d loss: %.3f (70, 0.42041999804519775)
Epoch %d loss: %.3f (80, 0.41136714656007467)
Epoch %d loss: %.3f (90, 0.40134328327038493)
Epoch %d loss: %.3f (100, 0.3902227392324411)
Epoch %d loss: %.3f (110, 0.37786443547696913)
Epoch %d loss: %.3f (120, 0.36419404986784165)
Epoch %d loss: %.3f (130, 0.34931043401364625)
Epoch %d loss: %.3f (140, 0.3335656502267289)
Epoch %d loss: %.3f (150, 0.3175524598795173)
Epoch %d loss: %.3f (160, 0.3019733678486572)
Epoch %d loss: %.3f (170, 0.28745273933991766)
Epoch %d loss: %.3f (180, 0.274400173670289)
Epoch %d loss: %.3f (190, 0.2629823600871015)
Epoch %d loss: %.3f (200, 0.2531770649502129)
Epoch %d loss: %.3f (210, 0.2448513